# 아래 데이터를 RDD로 만들고, 성적의 합계 및 평균을 계산하세요.

문항별로 RDD를 만들지 말고, 하나의 RDD만을 생성하고, 이를 변형해서 문제를 푸세요.



이름 | 과목 | 점수

-----|-----|-----

김하나 | English | 100

김하나 | Math | 80

임하나 | English | 70

임하나 | Math | 100

김갑돌 | English | 82.3

김갑돌 | Math | 98.5

# RDD 만들기


In [6]:
import pyspark;

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()
marks=[
    "'김하나','English', 100",
    "'김하나','Math', 80",
    "'임하나','English', 70",
    "'임하나','Math', 100",
    "'김갑돌','English', 82.3",
    "'김갑돌','Math', 98.5"
]
_marksRdd=spark.sparkContext.parallelize(marks)

* 문제 3-1: 이름으로 합계를 구해보자. 올바른 출력은 다음과 같다.



```

'임하나' 170.0

'김하나' 180.0

'김갑돌' 180.8

```

In [11]:
_marksbyname=_marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[0],float(x[2])))\
    .reduceByKey(lambda x,y:x+y)\
    .collect()

for i in _marksbyname:
    print(i[0],i[1])

'김하나' 180.0
'임하나' 170.0
'김갑돌' 180.8


* 문제 3-2: 과목으로 합계를 계산해 보자. 출력은 다음과 같이 나와야 한다.



```

'English' 252.3

'Math' 278.5

```

In [13]:
_marksbysubject=_marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[1],float(x[2])))\
    .reduceByKey(lambda x,y:x+y)\
    .collect()
for i in _marksbysubject:
    print (i[0],i[1])

'English' 252.3
'Math' 278.5


* 문제 3-3: 이름으로 합계과 개수를 구해보자. 출력은 다음과 같이 계산된다.



```

'임하나' (170.0, 2)

'김하나' (180.0, 2)

'김갑돌' (180.8, 2)

```

In [15]:
_marksbyname2=_marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[0],float(x[2])))

In [1]:
# 데이터 형태, 합계 개수, 머지  
sum_counts = _marksbyname2.combineByKey(
    (lambda x: (x, 1)), # initial value
    (lambda acc, value: (acc[0]+value, acc[1]+1)), # 키 맞는 것들 어떻게 합칠 것인가
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) # 계산 한것들 합침
)

NameError: name '_marksbyname2' is not defined

In [17]:
for i in sum_counts.collect():
    for each in i:
        print (each, end=' ')
    print()

'김하나' (180.0, 2) 
'임하나' (170.0, 2) 
'김갑돌' (180.8, 2) 


* 문제 3-4: 이름으로 평균을 계산해 보자. 앞서 3-3에서 사용했던 결과를 활용하고, 올바른 출력은 다음과 같다.



```

'임하나' 85.0

'김하나' 90.0

'김갑돌' 90.4

```



* 제출: ipynb 파일 1개 (zip하지 마세요).

In [18]:
# average
averageByKey = sum_counts\
    .map(lambda x: (x[0],x[1][0]/x[1][1]))\
    .collect()
for i in averageByKey:
    for j in i:
        print (j, end=' ')
    print()

'김하나' 90.0 
'임하나' 85.0 
'김갑돌' 90.4 
